In [ ]:
## SEQUENCING SPARE PARTS

In [ ]:
SENSOR_RECEPTIVE_FIELD_SIZE = 8
SENSOR_RECEPTIVE_FIELD_SQUARE = SENSOR_RECEPTIVE_FIELD_SIZE ** 2

SENSORS_COUNT = config.sensors_count
SENSORS_COUNT_ROOT = int(np.sqrt(SENSORS_COUNT))
assert SENSORS_COUNT_ROOT ** 2 == SENSORS_COUNT
assert SENSORS_COUNT_ROOT > 1

In [ ]:
SENSOR_STATES_INFO = defaultdict(list)
SENSOR_STATES_IMG = []
sz = SENSOR_RECEPTIVE_FIELD_SIZE
captions = []

assert config.encoding_type == 'sequence'

angle_step = 10 # degrees
angles = list(range(0, 360, angle_step)) + [45 * 1, 45 * 3, 45 * 5, 45 * 7]
angles = sorted(angles)
sweeps = [90, 130, 180, 230, 270] # for receptive field size = 8 more finer sweeps make little sense 

for angle in angles:
    if angle == 0:
        # add uber sensor state which will respond to completely illuminated patch
        white_canvas = Image.new('L', (sz, sz), color=255)
        captions.append(f'ang={angle}, UBER')
        SENSOR_STATES_IMG.append(white_canvas)
        SENSOR_STATES_INFO['normal'].append(angle)
        SENSOR_STATES_INFO['sweep'].append(0)
        SENSOR_STATES_INFO['normal_vec'].append(0j)
        SENSOR_STATES_INFO['sweep_vec'].append(0j)
        SENSOR_STATES_INFO['sweep1_vec'].append(0j)
        SENSOR_STATES_INFO['sweep2_vec'].append(0j)
    
    for sweep in sweeps:
        canvas = Image.new('L', (sz, sz))
        draw = ImageDraw.Draw(canvas)
        draw.rectangle([0, 0, sz, sz], fill=127, outline=127)
        piesclice_angle1 = angle + sweep // 2
        piesclice_angle2 = angle - sweep // 2
        draw.pieslice([-sz, -sz, sz * 2 - 1, sz * 2 - 1], start=piesclice_angle1, end=piesclice_angle2, fill=255, outline=255, width=0)
        captions.append(f'ang={angle}, swp={sweep} ')
        SENSOR_STATES_IMG.append(canvas)
        SENSOR_STATES_INFO['normal'].append(angle)
        SENSOR_STATES_INFO['sweep'].append(sweep)
        SENSOR_STATES_INFO['normal_vec'].append(np.exp((1j) * np.radians(angle)))
        SENSOR_STATES_INFO['sweep_vec'].append(np.exp((1j) * np.radians(sweep)))
        SENSOR_STATES_INFO['sweep1_vec'].append(np.exp((1j) * np.radians(piesclice_angle1)))
        SENSOR_STATES_INFO['sweep2_vec'].append(np.exp((1j) * np.radians(piesclice_angle2)))

SENSOR_STATES_INFO = pd.DataFrame(SENSOR_STATES_INFO)
assert len(SENSOR_STATES_INFO) == len(SENSOR_STATES_IMG)
# n_to_show = len(sweeps) * 4
# images_to_show = list(map(lambda x: lay_grid(x.resize((80, 80)), 8), SENSOR_STATES_IMG[:n_to_show]))
# display_images_grid(images_to_show, captions=captions[:n_to_show], col_count=len(sweeps) )

In [ ]:
# Turn grayscale images of sensor states to numpy arrays with weights which balance positive and negative areas
SENSOR_STATES = np.array(list(map(np.array, SENSOR_STATES_IMG))).astype(float)
SENSOR_STATES = SENSOR_STATES.reshape(-1, SENSOR_RECEPTIVE_FIELD_SIZE ** 2)
SENSOR_STATES[SENSOR_STATES == 255] = 1
SENSOR_STATES[SENSOR_STATES == 127] = -1
assert set(np.unique(SENSOR_STATES)) == set([-1, +1])

SENSOR_STATES_COUNT = SENSOR_STATES.shape[0]
# SENSOR_STATES.shape, np.unique_counts(SENSOR_STATES)

In [ ]:
SENSOR_STATE_AREAS_POS = (SENSOR_STATES == 1).sum(axis=1) 
SENSOR_STATE_AREAS_NEG = (SENSOR_STATES == -1).sum(axis=1)
assert SENSOR_STATE_AREAS_NEG[0] == 0
SENSOR_STATE_AREAS_NEG[0] = -1 # UBER sensor state
assert np.all(SENSOR_STATE_AREAS_POS > 0)
assert np.all(SENSOR_STATE_AREAS_NEG[1:] > 0)
# SENSOR_STATE_AREAS_POS.shape, SENSOR_STATE_AREAS_POS.mean(), SENSOR_STATE_AREAS_POS.std(), \
# SENSOR_STATE_AREAS_NEG.shape, SENSOR_STATE_AREAS_NEG.mean(), SENSOR_STATE_AREAS_NEG.std()

In [ ]:
SENSOR_STATES_POS = xp_array_to_gpu_copy(SENSOR_STATES)
SENSOR_STATES_NEG = xp_array_to_gpu_copy(SENSOR_STATES)
SENSOR_STATES_POS[SENSOR_STATES_POS < 0] = 0
SENSOR_STATES_NEG[SENSOR_STATES_NEG > 0] = 0

In [ ]:
class SensorLayoutMaker(object):
    def __call__(self, sensor_index):
        return None

class SensorLayoutMaker_Grid(SensorLayoutMaker):
    def __init__(self):
        self.s = SENSORS_COUNT_ROOT
        self.field_size = SENSOR_RECEPTIVE_FIELD_SIZE
        space_for_sensors = self.s * self.field_size 
        self.dist_between_sensors = (config.sample_size - space_for_sensors) / (self.s - 1)
        
    def __call__(self, sensor_index):
        x = int((sensor_index % self.s) * (self.field_size + self.dist_between_sensors))
        y = int((sensor_index // self.s) * (self.field_size + self.dist_between_sensors))
        # adjust to fit into boundaries
        x -= max(0, (x + self.field_size) - config.sample_size) 
        y -= max(0, (y + self.field_size) - config.sample_size)
        return x, y, self.field_size

In [ ]:
SENSOR_LAYOUT_MAKER = None
    
match config.retina_layout:
    case 'grid': 
        SENSOR_LAYOUT_MAKER = SensorLayoutMaker_Grid()
    case _:
        assert False, config.retina_layout

In [ ]:
assert not SENSOR_LAYOUT_MAKER is None
SENSOR_INSTANCES_INFO = pd.DataFrame(columns=['x_offset', 'y_offset', 'size', 'bounds_rect', 'x_center', 'y_center', 'radius'])

for i in range(SENSORS_COUNT):
    x_offset, y_offset, field_size = SENSOR_LAYOUT_MAKER(i)
    assert x_offset >= 0 and x_offset <= config.sample_size - field_size, (i, x_offset, field_size)
    assert y_offset >= 0 and y_offset <= config.sample_size - field_size, (i, y_offset, field_size)
    SENSOR_INSTANCES_INFO.loc[len(SENSOR_INSTANCES_INFO)] = [
        x_offset,
        y_offset,
        field_size,
        [x_offset, y_offset, x_offset + field_size - 1, y_offset + field_size - 1],
        x_offset + field_size // 2,
        y_offset + field_size // 2,
        field_size // 2
    ]

# SENSOR_INSTANCES_INFO

In [ ]:
SENSOR_PATCH_COORDS = []
    
for row in SENSOR_INSTANCES_INFO.itertuples():
    x_offset = row.x_offset
    y_offset = row.y_offset
    size = row.size

    for i in range(size):
        y = (y_offset + i) * config.sample_size
        stride_coords = np.arange(y + x_offset, y + x_offset + size)
        SENSOR_PATCH_COORDS.append(stride_coords)

SENSOR_PATCH_COORDS = np.array(SENSOR_PATCH_COORDS)

In [ ]:
def get_image_patches_for_sensor_instances(images_matrix):
    assert images_matrix.ndim == 2, images_matrix.ndim
    assert images_matrix.shape[1] == config.sample_size ** 2, images_matrix.shape[1]
    images_count = images_matrix.shape[0]
    # Result is 3d-tensor: 1 dim - image, 2 dim - patch no, 3 dim - patch itself (pixels)
    return np.take(images_matrix, SENSOR_PATCH_COORDS, axis=1).reshape(images_count, -1, SENSOR_RECEPTIVE_FIELD_SQUARE)

In [ ]:
# Returns matrix of sense_vectors (one row = sense_vector for each image in images_matrix)
# Each elem in sense_vector describes sensor state of each sensor instance (-1 if sensor instance is not activated)
def sense_images(images_matrix):
    BRIGHT_ILLUMINATION_ABS_LEVEL = 255 * 0.5
    BRIGHT_ILLUMINATION_DIRST_REL_DIFF = 0.75
    MINIMAL_ILLUMINATION_ABS_LEVEL = 255 * 0.1
    MINIMAL_ILLUMINATION_DIFF_DB = 2 # dB
    MINIMAL_ILLUMINATION_DIFF_RATIO = pow(10, MINIMAL_ILLUMINATION_DIFF_DB/20)
    MINIMAL_PLUS_MINUS_DISTR_REL_DIFF = 0.9
    MIN_COMBO_SCORE_VALUE = 1 + 1 + (MINIMAL_PLUS_MINUS_DISTR_REL_DIFF * 2) # 1 for abs illum MAC, 1 for diff abs illum mac, the last one is for +/- tallying
    
    assert images_matrix.ndim == 2
    images_count = images_matrix.shape[0]
    images_patches = get_image_patches_for_sensor_instances(images_matrix) # array of matrices: image no -> matrix with patches as row vectors
    
    mean_luminiscene_in_patches = images_patches.mean(axis=2) # matrix where each row = avg of luminisnece in each patch of an image
    mean_images_patches = images_patches.reshape(-1, SENSOR_RECEPTIVE_FIELD_SQUARE).T - mean_luminiscene_in_patches.ravel()
    mean_images_patches = mean_images_patches.T.reshape(images_count, -1, SENSOR_RECEPTIVE_FIELD_SQUARE).transpose(0, 2, 1)
    # mean_images_patches is an array of matrices: image no -> matrix with mean-centered patches as column vectors

    min_luminiscene_in_patches = images_patches.min(axis=2) # matrix where each row = minimal of luminiscence in each patch of an image
    min_images_patches = images_patches.reshape(-1, SENSOR_RECEPTIVE_FIELD_SQUARE).T.copy()
    assert min_images_patches.base is None # make sure we don't corrupt images_patches during next assignment
    min_images_patches[:] = min_luminiscene_in_patches.ravel()
    min_images_patches = min_images_patches.T.reshape(images_count, -1, SENSOR_RECEPTIVE_FIELD_SQUARE).transpose(0, 2, 1)
    # min_images_patches is an array of matrices: image no -> matrix with patches of minimal luminiscence as column vectors
    
    images_patches = images_patches.transpose(0, 2, 1) # array of matrices: image no -> matrix with patches as column vectors
    assert images_patches.shape == mean_images_patches.shape
    assert images_patches.shape == min_images_patches.shape
    
    abs_illum_level_pos = (SENSOR_STATES_POS @ images_patches).transpose(0, 2, 1)
    abs_illum_level_neg = (SENSOR_STATES_NEG @ images_patches).transpose(0, 2, 1)
    rel_illum_level_pos = (SENSOR_STATES_POS @ xp.where(mean_images_patches > 0, +1, 0)).transpose(0, 2, 1) # for +/- tally
    rel_illum_level_neg = (SENSOR_STATES_NEG @ xp.where(mean_images_patches < 0, -1, 0)).transpose(0, 2, 1)  # for +/- tally
    min_illum_level_pos = (SENSOR_STATES_POS @ min_images_patches).transpose(0, 2, 1)
    
    assert abs_illum_level_pos.shape == (images_count, SENSORS_COUNT, SENSOR_STATES_COUNT), abs_illum_level_pos.shape
    assert abs_illum_level_neg.shape == abs_illum_level_pos.shape
    assert rel_illum_level_pos.shape == abs_illum_level_pos.shape
    assert rel_illum_level_neg.shape == abs_illum_level_pos.shape
    assert min_illum_level_pos.shape == abs_illum_level_pos.shape
    
    sense_vectors = []
    # For each patch in image we detect likelihood of activation of sensor in one of its (sensor's) state.
    # If we have some state (say some angle and some sweep), what is condition that given patch corresponds to this state?
    # Of course patch must be somewhat illuminated (regulated by SENSORS_POS) - this is to ensure that we don't get fooled by situation
    # when e.g. negative part is not illuminated (dot prod is 0) while positive part is illuminated very, very week (say dot prod is 5). Looks
    # like this is the reason why we don't hear very low frequencies or don't percieve low light - becase ratio get oversaturaed very quickly.
    # But the key point is that sum of pixels luminiscence can meet averaged criterion only when bright pixels are concentared in area covered by SENSORS_POS
    # and dark pixels are concentrated in area covered by SENSORS_NEG. Otherwise there will disbalance.
    # To detect this condition we can look at distribution of mean-centered patches: +pixels must reside in positive part, -pixels must reside in negative part.
    # To restassure that +/- distribution of pixels actually corresponds to detectable diff in illumination we also need to compare
    # absolute illumination on positive and negative part
    for ailp_i, ailn_i, rilp_i, riln_i, milp_i in zip(abs_illum_level_pos, abs_illum_level_neg, rel_illum_level_pos, rel_illum_level_neg, min_illum_level_pos): # Per image cycle
        # Here and later on "patches count" 1-1 correspond to "sensor instances count" (for each sensor instance we prepare unique patch)
        # ailp_i and ailn_i both are matrices N(patches count) x M(count of sensor states)
        shape_save = ailp_i.shape

        # UBER - must be illuminated brightly and uniformly. I.e. 1) abs illum level (sum or products of SENSOR_STATE_POS X abs illum patches) must exceed bright criteria
        # 2) min illum level (sum of products of SENSOR_STATE_POS X min illum patches) must be close to abs illum level (this way we control minimal spread in illumination)
        bright_abs_illumination_scores = ailp_i > BRIGHT_ILLUMINATION_ABS_LEVEL * SENSOR_STATE_AREAS_POS
        uniform_illumination_scores = (milp_i / (ailp_i + 1e-6)) > BRIGHT_ILLUMINATION_DIRST_REL_DIFF
    
        # MAC {0, 1}
        # Illumination level on positive area must exceed lower threshold (sensor must be somewhat illuminated)
        # abs_illumination_scores = matrix of N(patches count), columns - 0/1 if patch is minimally illuminated according to given sensor's state
        abs_illumination_scores = ailp_i > MINIMAL_ILLUMINATION_ABS_LEVEL * SENSOR_STATE_AREAS_POS
    
        # MAC {0, 1}
        # Illumination level on positive part must distinguishably exceed illumination level of negative part
        ailn_i += 1e-6 # to get rid of possible division by zero errors
        ailp_i = ailp_i / SENSOR_STATE_AREAS_POS
        ailn_i = ailn_i / SENSOR_STATE_AREAS_NEG
        diff_abs_illumination_scores = xp.abs(ailp_i.ravel() / ailn_i.ravel())
        diff_abs_illumination_scores = diff_abs_illumination_scores >= MINIMAL_ILLUMINATION_DIFF_RATIO
        # diff_abs_illumination_scores = matrix of N(patches count), columns - 0/1 if patch is illuminated more on positive part than on negative
        diff_abs_illumination_scores = diff_abs_illumination_scores.reshape(shape_save)
    
        # Score {0, [MINIMAL_PLUS_MINUS_DISTR_REL_DIFF...1]}
        # Tally concentration of +/- pixels on positive/negative side and compare against theirs areas. 
        # For sensors with proper distribution ratio will be around 1 for each of the scores.
        # *concentration_scores = matrix of N(patches count), columns - [0...1] how well pixels in patch are distributed according to given sensor's state
        pluses_concentration_scores = rilp_i / SENSOR_STATE_AREAS_POS
        # assert xp.all((pluses_concentration_scores >= 0) & (pluses_concentration_scores <= 1)) # comment for production (to speed up)
        mask = (pluses_concentration_scores < MINIMAL_PLUS_MINUS_DISTR_REL_DIFF)
        pluses_concentration_scores = xp.where(mask, 0, pluses_concentration_scores)
    
        minuses_concentration_scores = riln_i / SENSOR_STATE_AREAS_NEG
        # assert xp.all((minuses_concentration_scores >= 0) & (minuses_concentration_scores <= 1)) # comment for production (to speed up)
        mask = (minuses_concentration_scores < MINIMAL_PLUS_MINUS_DISTR_REL_DIFF)
        minuses_concentration_scores = xp.where(mask, 0, minuses_concentration_scores)

        # sense_vector_uber = vector of N(patches count), each value - logical sum of UBER MACs for first sensor state 
        sense_vector_uber = bright_abs_illumination_scores[:,0] & uniform_illumination_scores[:,0]
        sense_vector_uber = xp.where(sense_vector_uber, 0, -1) # True -> 0 (UBER sensor state), False -> -1
        
        # sense_matrix = matrix of N(patches count), columns - sum of MACs and scores for given sensor state
        sense_matrix = (abs_illumination_scores.astype(float) + diff_abs_illumination_scores.astype(float) + pluses_concentration_scores + minuses_concentration_scores)
        # take all columns except the first one. First column is an UBER sensor state intended for detection of completely illuminated patch. 
        # UBER sensor state is evaluated if other sensor states didn't respond.
        sense_submatrix = sense_matrix[:,1:] 
        sense_vector = xp.where(xp.any(sense_submatrix > MIN_COMBO_SCORE_VALUE, axis=1), 1 + xp.argmax(sense_submatrix, axis=1), -1)
        sense_vector = xp.where(sense_vector == -1, sense_vector_uber, sense_vector)
        sense_vectors.append(sense_vector)
    
    return xp.vstack(sense_vectors)

In [ ]:
# namedtuple for data export
SensorInstance = namedtuple('SensorInstance', ['Index', 'x', 'y', 'normal', 'normal_vec', 'sweep', 'sweep_vec', 'sweep1_vec', 'sweep2_vec', 'x2', 'y2'])

In [ ]:
SequencingContext = namedtuple('SequencingContext', 
                               ['altitude_map_hires', 
                                'altitude_lores_to_hires_translate_factor', 
                                'si_list', 
                                'outer_grooves', 
                                'outer_si_ind_dict',
                                'inner_grooves',
                                'inner_si_ind_dict',
                                'outer_raw_sequences',
                                'inner_raw_sequences', 
                                'outer_sequences',
                                'inner_sequences'],
                              defaults = [None] * 9)

In [ ]:
ALTITUDE_INNER = 3 #  value to designate inner interior of any figure

# Returns (si_list, altitude_map) for sense_vector
def parse_sense_vector(sense_vector):
    ALTITUDE_EXTEND_BY = 5 # number of new intermediate cells for each side of 2x2 rect, e.g. 3x3 becomes 13x13 if altitude_extend_by is 5

    assert sense_vector.shape[0] == len(SENSOR_INSTANCES_INFO)
    sensor_instance_inds = np.argwhere(sense_vector != -1).ravel()
    sensor_state_inds = np.take(sense_vector, sensor_instance_inds)
    assert sensor_instance_inds.shape[0] == sensor_state_inds.shape[0]
    df_sensor_instances_uberraw = pd.DataFrame({'sensor_instance_ind': sensor_instance_inds, 'sensor_state_ind': sensor_state_inds})
    df_sensor_instances_uberraw = df_sensor_instances_uberraw.merge(SENSOR_STATES_INFO, how='left', left_on='sensor_state_ind', right_index=True)
    df_sensor_instances_uberraw = df_sensor_instances_uberraw.merge(SENSOR_INSTANCES_INFO, how='left', left_on='sensor_instance_ind', right_index=True)
    df_sensor_instances_uberraw.rename({'x_center': 'x', 'y_center': 'y'}, axis=1, inplace=True, errors='raise')
    df_sensor_instances_uberraw[['x2', 'y2']] = df_sensor_instances_uberraw[['x', 'y']]
    df_sensor_instances_raw = df_sensor_instances_uberraw.loc[df_sensor_instances_uberraw.sensor_state_ind != 0].copy()
    df_sensor_instances_raw.drop(['sensor_instance_ind', 'sensor_state_ind', 'bounds_rect', 'sweep_vec', 'sweep1_vec', 'sweep2_vec', 'radius'], axis=1, inplace=True)
    si_list_uberraw = list(df_sensor_instances_uberraw.itertuples())
    si_list_raw = list(df_sensor_instances_raw.itertuples())

    # Lo-resolution altitude map
    altitude_map_lores = np.zeros((config.sample_size, config.sample_size))

    for si in si_list_uberraw:
        sensor_state = SENSOR_STATES[si.sensor_state_ind]
        size = si.size
        altitude_map_lores[si.y_offset:si.y_offset + size, si.x_offset:si.x_offset + size] += sensor_state.reshape(size, size)
    
    min_altitude = np.min(altitude_map_lores)
    altitude_map_si_abscence_mask = np.full_like(altitude_map_lores, min_altitude, dtype='i')
    
    for si in si_list_uberraw:
        altitude_map_si_abscence_mask[si.y_offset:si.y_offset + size, si.x_offset:si.x_offset + size] = 0
    
    altitude_map_lores += altitude_map_si_abscence_mask
    altitude_map_lores = altitude_map_lores.astype('i')

    # Enhance lo-resolution altitude map by interpolating. This is to develop low altitude areas which are 
    # hidden beyond patches like [[-4, -5], [3, 4]]
    altitude_lores_to_hires_translate_factor = (ALTITUDE_EXTEND_BY + 1)
    altitude_map_hires_size = (config.sample_size - 1) * altitude_lores_to_hires_translate_factor + 1
    altitude_map_hires = np.zeros((altitude_map_hires_size, altitude_map_hires_size), dtype='f')
    patch_hires_shape = (2 + ALTITUDE_EXTEND_BY, 2 + ALTITUDE_EXTEND_BY)
    ii, jj = np.mgrid[0:patch_hires_shape[0], 0:patch_hires_shape[1]]
    patch_hires_indices = np.array(list(zip(ii.ravel(), jj.ravel())))
    
    for i in range(altitude_map_lores.shape[0] - 1):
        for j in range(altitude_map_lores.shape[1] - 1):
            patch = altitude_map_lores[i:i+2, j:j+2].astype(float)
    
            if patch[0,0] == patch[0,1] and patch[0,1] == patch[1,0] and patch[1,0] == patch[1,1]:
                patch_hires = patch[0,0]
            else:
                interp = RegularGridInterpolator(([0, patch_hires_shape[0] - 1], [0, patch_hires_shape[1] - 1]), patch)
                patch_hires = interp(patch_hires_indices).reshape(patch_hires_shape)
                
            i_hires = i * altitude_lores_to_hires_translate_factor
            j_hires = j * altitude_lores_to_hires_translate_factor
            altitude_map_hires[i_hires:i_hires+patch_hires_shape[0], j_hires:j_hires+patch_hires_shape[1]] = patch_hires
    
    altitude_map_hires = altitude_map_hires.astype('i')
    altitude_map_hires[altitude_map_hires > ALTITUDE_INNER - 1] = ALTITUDE_INNER

    # Reposition sensor instances so they occur on a groove which surrounds outer
    # contour of image parts. Some of the sensor instances will get discarded
    si_list = []
    xy_unique = set()
    
    for si in si_list_raw:
        x = int(si.x * altitude_lores_to_hires_translate_factor)
        y = int(si.y * altitude_lores_to_hires_translate_factor)
        altitude = altitude_map_hires[y, x]
        is_top_altitude_reached = False
        travel_point = complex(x, y)
    
        for _ in range(100): # fail fast on too many iterations (shouldn't happen often)
            x = int(travel_point.real)
            y = int(travel_point.imag)
    
            if x >= 0 and x < altitude_map_hires.shape[1] and y >= 0 and y < altitude_map_hires.shape[0]:
                if altitude_map_hires[y, x] >= ALTITUDE_INNER:
                    is_top_altitude_reached = True
                    break
            else:
                # we've hit image boundaries, ignore this si (alternative implementation may be possible though)
                break
                
            travel_point -= si.normal_vec # crawl in direction opposite to normal
    
        if not is_top_altitude_reached:
            continue
    
        is_cliff_reached = False
            
        for _ in range(100):
            x = int(travel_point.real)
            y = int(travel_point.imag)
    
            if x >= 0 and x < altitude_map_hires.shape[1] and y >= 0 and y < altitude_map_hires.shape[0]:
                if altitude_map_hires[y, x] < ALTITUDE_INNER:
                    # This si falled out of cliff
                    break
            
            for rel_xy in itertools.product(range(-1, 1 + 1), repeat=2):
                test_x = x + rel_xy[0]
                test_y = y + rel_xy[1]
        
                if test_x < 0 or test_y < 0 or test_x >= altitude_map_hires.shape[1] or test_y >= altitude_map_hires.shape[0]:
                    continue
                elif altitude_map_hires[test_y, test_x] >= ALTITUDE_INNER:
                    continue
                
                is_cliff_reached = True
                break
            else:
                travel_point += si.normal_vec
    
            if is_cliff_reached:
                break
                        
        if is_cliff_reached:
            if not (x, y) in xy_unique:
                # Make sure NO xy lays on the edge of altitude map boundaries (otherwise we will face with constant checks and code will be complicated/slowdown)
                assert x > 0 and x < altitude_map_hires.shape[1] - 1
                assert y > 0 and y < altitude_map_hires.shape[0] - 1
                x_lores = round(x / altitude_lores_to_hires_translate_factor)
                y_lores = round(y / altitude_lores_to_hires_translate_factor)
                si_list.append(si._replace(x2=x, y2=y, x=x_lores, y=y_lores))
                xy_unique.add((x, y))

    return SequencingContext(si_list=si_list, altitude_map_hires=altitude_map_hires, altitude_lores_to_hires_translate_factor=altitude_lores_to_hires_translate_factor)

In [ ]:
NEAREST_NEIGHB_REL_XY_LIST = list(itertools.product(range(-1, 1 + 1), repeat=2))
NEAREST_NEIGHB_REL_XY_LIST.remove((0, 0))
NEAREST_NEIGHB_REL_XY_LIST = np.array(NEAREST_NEIGHB_REL_XY_LIST)

In [ ]:
def get_outer_grooves(seq_context):
    wave_fronts = deque()
    xy_touched = set()
    
    for si_ind, si in enumerate(seq_context.si_list):
        x, y = (si.x2, si.y2)
        assert not (x, y) in xy_touched
        xy_touched.add((x, y))
        wave_fronts.append((x, y, si_ind))
        
    outer_grooves_map = np.full_like(seq_context.altitude_map_hires, -1, dtype='i')
    
    while wave_fronts:
        x, y, si_ind = wave_fronts.popleft()
        assert outer_grooves_map[y, x] == -1, outer_grooves_map[y, x]
    
        # point at x,y must be on cliff - i.e. one of its neighbours must be out of plateau (altitude=3)
        for rel_xy in NEAREST_NEIGHB_REL_XY_LIST:
            test_x = x + rel_xy[0]
            test_y = y + rel_xy[1]
    
            if seq_context.altitude_map_hires[test_y, test_x] < ALTITUDE_INNER:
                # cliff condition met
                break
        else:
            continue
        
        outer_grooves_map[y, x] = si_ind
    
        for rel_xy in NEAREST_NEIGHB_REL_XY_LIST:
            test_x = x + rel_xy[0]
            test_y = y + rel_xy[1]
    
            if seq_context.altitude_map_hires[test_y, test_x] < ALTITUDE_INNER:
                continue
            elif outer_grooves_map[test_y, test_x] != -1:
                continue
            elif (test_x, test_y) in xy_touched:
                continue
    
            wave_fronts.append((test_x, test_y, si_ind))
            xy_touched.add((test_x, test_y))
    
    outer_grooves = []
    
    for ij in np.argwhere(outer_grooves_map != -1):
        x, y = int(ij[1]), int(ij[0])
        si_ind = outer_grooves_map[y, x]
        outer_grooves.append((x, y, si_ind))
    
    outer_grooves = np.array(outer_grooves)
    outer_si_ind_dict = dict(map(lambda si_ind_si: (si_ind_si[0], si_ind_si[1]), enumerate(seq_context.si_list)))
    
    return seq_context._replace(outer_grooves=outer_grooves, outer_si_ind_dict=outer_si_ind_dict)

In [ ]:
def get_inner_grooves(seq_context):
    wave_fronts = deque()
    xy_touched = set()
    
    for x, y, si_ind in seq_context.outer_grooves:
        xy_touched.add((x, y))
        wave_fronts.append((x, y, si_ind))
    
    inner_grooves_map = np.full_like(seq_context.altitude_map_hires, -1, dtype='i')
    inner_grooves = []
    inner_groove_bits = []
    rel_xy_list = NEAREST_NEIGHB_REL_XY_LIST.copy()
    
    while wave_fronts:
        x, y, si_ind = wave_fronts.popleft()
        assert inner_grooves_map[y, x] == -1, inner_grooves_map[y, x]
        inner_grooves_map[y, x] = si_ind
        is_amberified = True
    
        RNG.shuffle(rel_xy_list)
        
        for rel_xy in rel_xy_list:
            test_x = x + rel_xy[0]
            test_y = y + rel_xy[1]
    
            # if test_x < 0 or test_y < 0 or test_x >= seq_context.altitude_map_hires.shape[1] or test_y >= seq_context.altitude_map_hires.shape[0]:
            #     continue
            # elif 
            if seq_context.altitude_map_hires[test_y, test_x] < ALTITUDE_INNER:
                continue
            elif inner_grooves_map[test_y, test_x] != -1:
                continue
            elif (test_x, test_y) in xy_touched:
                continue
    
            wave_fronts.append((test_x, test_y, si_ind))
            xy_touched.add((test_x, test_y))
            is_amberified = False
    
        if is_amberified:
            inner_groove_bits.append((x, y))
            si_ind_normal_vec = seq_context.si_list[si_ind].normal_vec
            
            for rel_xy in NEAREST_NEIGHB_REL_XY_LIST:
                test_x = x + rel_xy[0]
                test_y = y + rel_xy[1]
        
                # if test_x < 0 or test_y < 0 or test_x >= inner_grooves_map.shape[1] or test_y >= inner_grooves_map.shape[0]:
                #     continue
                # elif 
                if inner_grooves_map[test_y, test_x] == -1:
                    continue
                
                other_si_ind = inner_grooves_map[test_y, test_x]
    
                if si_ind == other_si_ind:
                    continue
                    
                other_si_ind_normal_vec = seq_context.si_list[other_si_ind].normal_vec
                dot_prod = si_ind_normal_vec.real * other_si_ind_normal_vec.real + si_ind_normal_vec.imag * other_si_ind_normal_vec.imag
    
                if dot_prod < 0: 
                    inner_grooves.append((x, y, si_ind))
                    break
    
    inner_grooves = np.array(inner_grooves)
    inner_si_ind_dict = {}
    
    for i in range(inner_grooves.shape[0]):
        x, y, si_ind = inner_grooves[i]
        si = seq_context.si_list[si_ind]
        x_lores = round(x / seq_context.altitude_lores_to_hires_translate_factor)
        y_lores = round(y / seq_context.altitude_lores_to_hires_translate_factor)
        new_si_ind_value = len(inner_si_ind_dict)
        inner_si_ind_dict[new_si_ind_value] = si._replace(x2=int(x), y2=int(y), x=x_lores, y=y_lores)
        inner_grooves[i, 2] = new_si_ind_value
    
    return seq_context._replace(inner_grooves=inner_grooves, inner_si_ind_dict=inner_si_ind_dict)

In [ ]:
GROOVE_ENDPOINT_TEMPLATE = np.array([[1, 1, 0], [1, 0, 0], [1, 1, 1]])
GROOVE_ENDPOINT_TEMPLATES = []

for t in [GROOVE_ENDPOINT_TEMPLATE, np.fliplr(GROOVE_ENDPOINT_TEMPLATE)]:
    GROOVE_ENDPOINT_TEMPLATES.append(t)
    GROOVE_ENDPOINT_TEMPLATES.append(np.rot90(t))
    GROOVE_ENDPOINT_TEMPLATES.append(np.rot90(np.rot90(t)))
    GROOVE_ENDPOINT_TEMPLATES.append(np.rot90(np.rot90(np.rot90(t))))

GROOVE_ENDPOINT_TEMPLATES = np.array(GROOVE_ENDPOINT_TEMPLATES)

def get_outer_starting_si_inds(grooves, grooves_map):
    # outer si_inds corresponds to a circular contour, 
    # so we can start anywhere (i.e. from any si)
    return set(grooves[:,2])

def get_inner_starting_si_inds(grooves, grooves_map):
    r = set()
    
    for x, y, si_ind in grooves:
        assert x > 0
        assert y > 0
        assert x < grooves_map.shape[1] - 1
        assert y < grooves_map.shape[0] - 1
        patch = grooves_map[y-1:y+2, x-1:x+2]
        s = (GROOVE_ENDPOINT_TEMPLATES * patch).reshape(GROOVE_ENDPOINT_TEMPLATES.shape[0], -1).sum(axis=-1)
        
        if np.any(s == -6):
            r.add(si_ind)

    return r

In [ ]:
def _populate_raw_sequences(raw_sequences, seq_context, grooves, si_ind_dict, get_starting_si_inds):
    grooves_map = np.full_like(seq_context.altitude_map_hires, -1, dtype='i')

    for x, y, si_ind in grooves:
        grooves_map[y, x] = si_ind
    
    xy_on_grooves_to_si_ind = dict(map(lambda item: ((item[1].x2, item[1].y2), item[0]), si_ind_dict.items()))  # xy -> si_ind. Deduplication by xy happens here
    starting_si_inds = get_starting_si_inds(grooves, grooves_map)
    
    while starting_si_inds:
        # choose random sensor instance to start from
        start_sensor_inst_index = starting_si_inds.pop()
        si = si_ind_dict[start_sensor_inst_index]
        si_xy = (si.x2, si.y2)
        si_ij = int(si_xy[1]), int(si_xy[0])
        
        wave_front_que = deque([si_ij])
        wave_front_map = np.full_like(seq_context.altitude_map_hires, -1, dtype='i')
        wave_front_map[wave_front_que[0]] = start_sensor_inst_index
        wave_front_touched = set([wave_front_que[0]])
        
        sensor_inst_chains = {} # key - last element in chain of sensor instances, value - chain itself
        
        # LOG(f'Starting walk from {start_sensor_inst_index} (of {len(starting_si_inds)})')
        
        while wave_front_que:
            # Visit point on top of the wave_front
            wave_front_point = wave_front_que.popleft()
            wave_front_sensor_inst_ind = int(wave_front_map[wave_front_point])
            wave_front_sensor_inst_ind1 = wave_front_sensor_inst_ind
            wave_front_map[wave_front_point] = -1
            assert wave_front_sensor_inst_ind > -1
            # LOG(f'Visiting {wave_front_point} (si={wave_front_sensor_inst_ind}) (of {len(wave_front_que)})')
        
            si_ind = xy_on_grooves_to_si_ind.get((wave_front_point[1], wave_front_point[0]), -1)
        
            if si_ind != -1 and si_ind != wave_front_sensor_inst_ind:
                # New sensor instance discovered! Link with the previous one, ...
                sensor_inst_chain = sensor_inst_chains.pop(wave_front_sensor_inst_ind, [wave_front_sensor_inst_ind])
                sensor_inst_chain.append(si_ind)
                assert not si_ind in sensor_inst_chains, f'{si_ind} in sensor_inst_chains'
                sensor_inst_chains[si_ind] = sensor_inst_chain
                
                # LOG(f'New si encountered: {si_ind}, linked after {wave_front_sensor_inst_ind}')
                wave_front_sensor_inst_ind = si_ind
                # ... and adjust adjacent wave front points to account for new sensor instance
                # (adjacency criteria enables coexisting of separate (disjoint) wave fronts on a map)
                adjust_stack = [wave_front_point]
                adjust_touched = set([wave_front_point])
        
                while adjust_stack:
                    adjust_point = adjust_stack.pop()
        
                    for rel_ij in [[0, -1], [-1, 0], [0, 1], [1, 0], [-1, -1], [-1, 1], [1, 1], [1, -1]]:
                        ij = int(adjust_point[0] + rel_ij[0]), int(adjust_point[1] + rel_ij[1])
                
                        if wave_front_map[ij] == -1:
                            continue
                        elif ij in adjust_touched:
                            continue
        
                        # LOG(f'Adjusting {ij} (si={wave_front_map[ij]}) to si={si_ind}')
                        wave_front_map[ij] = si_ind
                        adjust_stack.append(ij)
                        adjust_touched.add(ij)
        
            # Plan further spread of the wave front
            for rel_ij in [[0, -1], [-1, 0], [0, 1], [1, 0], [-1, -1], [-1, 1], [1, 1], [1, -1]]:
                ij = int(wave_front_point[0] + rel_ij[0]), int(wave_front_point[1] + rel_ij[1])
                # LOG(f'... spread, checking coord {rel_ij} {ij}')
        
                if grooves_map[ij] == -1:  # coord_ij falls out of groove
                    # LOG(f'... spread, {ij} is out of groove')
                    continue
                elif ij in wave_front_touched:
                    # LOG(f'... spread, {ij} already touched ({len(wave_front_touched)})')
                    continue
        
                wave_front_touched.add(ij)
                wave_front_que.append(ij)
                wave_front_map[ij] = wave_front_sensor_inst_ind
                # LOG(f'... new wave front point {wave_front_que[-1]} (si={wave_front_sensor_inst_ind})')

        if sensor_inst_chains:
            raw_sequences[start_sensor_inst_index] = sensor_inst_chains
            
        # Collect all sensor instances encountered during this run and mark them as processed
        processed_sensor_instance_inds = set(itertools.chain.from_iterable(sensor_inst_chains.values()))
        starting_si_inds -= processed_sensor_instance_inds

In [ ]:
def get_outer_raw_sequences(seq_context):
    outer_raw_sequences = {}
    _populate_raw_sequences(outer_raw_sequences, seq_context, seq_context.outer_grooves, seq_context.outer_si_ind_dict, get_outer_starting_si_inds)
    return seq_context._replace(outer_raw_sequences=outer_raw_sequences)

In [ ]:
def get_outer_sequences(seq_context):
    outer_sequences = []
        
    for sensor_inst_chains in seq_context.outer_raw_sequences.values():
        # Our goal is to find two longest subsequence which start from the same si and clue them together. 
        # Usually this would be two subsequences starting from start_sensor_inst_index.
        # This way we would get longest serial sequence of traverse
        subsequences = []
        
        for chain_index, chain in enumerate(sensor_inst_chains.values()):
            # get all chains OTHER THAN current chain under chain_index
            chains_wo_chain_index = map(lambda v: v[1], filter(lambda v: v[0] != chain_index, enumerate(sensor_inst_chains.values())))
            chains_dict = defaultdict(list)
            for v in chains_wo_chain_index: chains_dict[v[0]].append(v)
            
            subsequences.append([])
            branch_stack = [(chain, 0, len(subsequences) - 1)]
            LOG(f'Starting new subsequence #{len(subsequences) - 1}, {chain[0]}')
        
            while branch_stack:
                chain, chain_elem_index, subseq_index = branch_stack.pop()
                subseq = subsequences[subseq_index]
                LOG(f'Weaving subsequence #{subseq_index}, {chain[chain_elem_index]}')
                
                for i in range(chain_elem_index, len(chain)):
                    v = chain[i]
                    subseq.append(v)
                    LOG(f'... adding {chain[i]}, subseq len={len(subseq)}, subseq[:3]={subseq[:3]}, subseq[-3:]={subseq[-3:]}')
        
                    if v in chains_dict:
                        LOG(f'... branch point detected')
                        # New branch point detected
                        branch_stack.append((chain, i + 1, subseq_index)) # continue sequence by following current chain
    
                        # Start new subsequences from new chain
                        for branch_chain in chains_dict[v]:
                            new_subseq = subseq.copy()
                            subsequences.append(new_subseq)
                            branch_stack.append((branch_chain, 1, len(subsequences) - 1)) # create new sequence and follow new chain
                            LOG(f'... branching new subsequence #{len(subsequences) - 1}, {branch_chain[1]}')
                            
                        break
    
        longest_sequence = []
    
        if not subsequences:
            continue
        elif len(subsequences) == 1:
            longest_subsequence = subsequences[0]
        else:
            for ii in itertools.combinations(range(len(subsequences)), 2):
                subseq1 = subsequences[ii[0]]
                subseq2 = subsequences[ii[1]]
                prefix_i = 0
        
                while prefix_i < min(len(subseq1), len(subseq2)) and subseq1[prefix_i] == subseq2[prefix_i]:
                    prefix_i += 1
        
                if prefix_i == 0:
                    # No common prefix, cant clue these two subseqs
                    continue
        
                sequence = list(reversed(subseq1[prefix_i:])) + [subseq1[prefix_i - 1]] + subseq2[prefix_i:]
        
                if len(sequence) > len(longest_sequence):
                    longest_sequence = sequence
    
        if longest_sequence:
            outer_sequences.append(longest_sequence)
    
    return seq_context._replace(outer_sequences=outer_sequences)

In [ ]:
def get_inner_raw_sequences(seq_context):
    inner_raw_sequences = {}
    _populate_raw_sequences(inner_raw_sequences, seq_context, seq_context.inner_grooves, seq_context.inner_si_ind_dict, get_inner_starting_si_inds)
    return seq_context._replace(inner_raw_sequences=inner_raw_sequences)

In [ ]:
def get_inner_sequences(seq_context):
    inner_sequences = list(itertools.chain.from_iterable(map(lambda chain_dict: chain_dict.values(), seq_context.inner_raw_sequences.values())))
    return seq_context._replace(inner_sequences=inner_sequences)